In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_csv('energy.csv')

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [4]:
df.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [5]:
model = LinearRegression()
model.fit(df[['T2']], df['T6'])

LinearRegression()

## Question 12 ##

In [6]:
model.score(df[['T2']], df['T6'])

0.6418990830855493

## Question 13

In [8]:
df = df.drop(['date', 'lights'], axis = 1)

In [55]:
scaler = MinMaxScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

x = df_norm.drop(['Appliances'], axis = 1)
y = df['Appliances']

x_train, x_test, y_train, y_test = train_test_split(x, y , test_size = 0.3 , random_state = 42)

lin_model = LinearRegression()
lin_model.fit(x_train, y_train)

prediction = lin_model.predict(x_test)

In [56]:
mean_absolute_error(prediction, y_test)

53.64297765584962

## Question 14

In [50]:
#Residual Sum of square

print(np.sum((y_test - prediction)**2))

51918501.21223275


## Question 15

In [51]:
from sklearn.metrics import mean_squared_error

rmse =np.sqrt(mean_squared_error(y_test, prediction))

round(rmse, 3)

93.64

## Question 16

In [54]:
#Coefficient of determination

from sklearn.metrics import r2_score

r2 = r2_score(y_test, prediction)

round(r2, 2)

0.15

## Question 17

In [31]:
x.columns

Index(['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5',
       'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out',
       'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1',
       'rv2'],
      dtype='object')

In [57]:
coeffs = pd.DataFrame({'feature':x_train.columns.to_list(), 'weights':lin_model.coef_})
coeffs.sort_values('weights')

,feature,weights
3,RH_2,-488.666805
18,T_out,-344.389845
2,T2,-252.710373
16,T9,-203.236627
15,RH_8,-168.627161
20,RH_out,-83.107599
13,RH_7,-47.736592
17,RH_9,-42.586339
8,T5,-16.752822
0,T1,-3.510725


## Question 18

In [64]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha = 0.4)

ridge_model.fit(x_train, y_train)

pred = ridge_model.predict(x_test)

rmse =np.sqrt(mean_squared_error(y_test, pred))

round(rmse, 3)

93.661

## Question 19

In [61]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha = 0.001)

lasso_model.fit(x_train, y_train)

pd.DataFrame({'feature':x_train.columns.to_list(), 'weights':lasso_model.coef_})

,feature,weights
0,T1,-4.147124e+00
1,RH_1,5.905998e+02
2,T2,-2.494688e+02
3,RH_2,-4.845653e+02
4,T3,3.102104e+02
5,RH_3,1.022046e+02
6,T4,3.034520e+01
7,RH_4,2.744590e+01
8,T5,-1.711872e+01
9,RH_5,1.710332e+01


In [63]:
'''
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = [ 'Features' , col_name]
    weights_df[col_name].round( 3 )
    return weights_df

linear_model_weights = get_weights_df(lin_model, x_train, 'Linear_Model_Weight' )
ridge_weights_df = get_weights_df(ridge_model, x_train, 'Ridge_Weight' )
lasso_weights_df = get_weights_df(lasso_model, x_train, 'Lasso_weight' )


final_weights = pd.merge(linear_model_weights, ridge_weights_df, on= 'Features' )
final_weights = pd.merge(final_weights, lasso_weights_df, on= 'Features' )
'''

"\ndef get_weights_df(model, feat, col_name):\n    weights = pd.Series(model.coef_, feat.columns).sort_values()\n    weights_df = pd.DataFrame(weights).reset_index()\n    weights_df.columns = [ 'Features' , col_name]\n    weights_df[col_name].round( 3 )\n    return weights_df\n\nlinear_model_weights = get_weights_df(lin_model, x_train, 'Linear_Model_Weight' )\nridge_weights_df = get_weights_df(ridge_model, x_train, 'Ridge_Weight' )\nlasso_weights_df = get_weights_df(lasso_model, x_train, 'Lasso_weight' )\n\n\nfinal_weights = pd.merge(linear_model_weights, ridge_weights_df, on= 'Features' )\nfinal_weights = pd.merge(final_weights, lasso_weights_df, on= 'Features' )\n"

In [41]:
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-488.666805,-439.846215,-4.845653e+02
1,T_out,-344.389845,-280.524329,-3.325717e+02
2,T2,-252.710373,-215.494498,-2.494688e+02
3,T9,-203.236627,-202.140437,-2.025025e+02
4,RH_8,-168.627161,-167.808155,-1.686955e+02
5,RH_out,-83.107599,-58.554305,-7.800039e+01
6,RH_7,-47.736592,-49.195346,-4.738909e+01
7,RH_9,-42.586339,-44.262196,-4.245510e+01
8,T5,-16.752822,-21.242949,-1.711872e+01
9,T1,-3.510725,-19.694643,-4.147124e+00


## Question 20

In [65]:
pre = lasso_model.predict(x_test)

rmse =np.sqrt(mean_squared_error(y_test, pre))

round(rmse, 3)

93.641